In [120]:
import argparse
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda, Conv2D
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body
import numpy as np
from tensorflow.keras.models import load_model, Model

In [62]:
def yolo_filter_boxes(box_confidence,boxes,box_class_probs,threshold = 0.6):
    
    box_scores = box_confidence*box_class_probs
    box_classes = K.argmax(box_scores,axis=-1)
    box_class_scores = K.max(box_scores,axis=-1,keepdims=False)
    
    filtering_mask = (box_class_scores>=threshold)
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes,filtering_mask)
    
    return scores,boxes,classes

In [76]:
with tf.Session() as test:
    
    box_confidence = tf.random_normal([19,19,5,1],mean=1,stddev=4,seed=1)
    boxes = tf.random_normal([19,19,5,4],mean=1,stddev=4,seed=1)
    box_class_probs = tf.random_normal([19,19,5,80],mean=1,stddev=4,seed=1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)
    print("scores[4] = " + str(scores[4].eval()))
    print("boxes[4] = " + str(boxes[4].eval()))
    print("classes[4] = "+str(classes[4].eval()))
    print(scores.shape)
    print(boxes.shape)
    print(classes.shape)
    


scores[4] = 12.057256
boxes[4] = [-5.9719086 -0.7515718 -1.26404    2.2826772]
classes[4] = 5
(?,)
(?, 4)
(?,)


In [75]:
scores.shape

TensorShape([Dimension(None)])

In [72]:
# Non-Max Suppression
# Intersection over Union


def iou(box1,box2):
    
    (box1_x1,box1_y1,box1_x2,box1_y2) = box1
    (box2_x1,box2_y1,box2_x2,box2_y2) = box2
    
    xi1 = np.maximum(box1[0],box2[0])
    yi1 = np.maximum(box1[1],box2[1])
    xi2 = np.minimum(box1[2],box2[2])
    yi2 = np.minimum(box1[3],box2[3])
    inter_width = xi2-xi1
    inter_height = yi2-yi1
    
#     case in which they doesn't intersect, then inter_height or inter_width will be 0 and intersection area will return 0

    intersection_area = max(inter_width,0)*max(inter_height,0)
    
#     Union_Area
    
    box1_area = (box1[2]-box1[0])*(box1[3]-box1[1])
    box2_area = (box2[2]-box2[0])*(box2[3]-box2[1])
    union_area = box1_area+box2_area-intersection_area
    
    iou = float(intersection_area)/float(union_area)
    
    
    return iou
    
    

In [73]:
# Test case run
box1 = (2,1,4,3)
box2 = (1,2,3,4)
print(iou(box1,box2))

0.14285714285714285


In [74]:
def non_max_suppression(scores,boxes,classes,max_boxes=10,iou_threshold=0.5):
    
    max_boxes_tensor =K.variable(max_boxes,dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    nms_indices = tf.image.non_max_suppression(boxes,scores,max_output_size=max_boxes,iou_threshold = iou_threshold)
    
    scores = tf.gather(scores,nms_indices)
    boxes = tf.gather(boxes,nms_indices)
    classes = tf.gather(classes,nms_indices)
    
    return scores, boxes, classes
    
    

In [92]:
with tf.Session() as test_1:
    scores = tf.random_normal([64,],mean=1,stddev=4,seed=1)
    boxes = tf.random_normal([64,4],mean=1,stddev=4,seed=1)
    classes = tf.random_normal([64,],mean=1,stddev=4,seed=1)
    
    scores,boxes,classes = non_max_suppression(scores,boxes,classes)
    
    print('Scores[2]=' + str(scores[2].eval()))
    print('boxes[2] = ' + str(boxes[2].eval()))
    print('classes[2] ='+str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " +str(boxes.eval().shape))
    print("classes.shape = " +str(classes.eval().shape))

Scores[2]=7.131255
boxes[2] = [-0.95718646  2.371214   10.73251     0.10136247]
classes[2] =2.4274013
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


In [93]:
# yolo_boxes_to_corners(box_xy, box_wh) converts the yolo box coordinates (x,y,w,h) to box corners
# coordinates (x1, y1, x2, y2) to fit the input of yolo_filter_boxes

# scale_boxes(boxes, image_shape) this step rescales the boxes so that they can be plotted on top of the original 720x1280 image

# YoLo Evaluation function

def yolo_eval(yolo_outputs,image_shape=(720.,1280.),max_boxes=10,score_threshold=0.6,iou_threshold=0.5):
    
    box_confidence, box_xy,box_wh,box_class_probs = yolo_outputs
    
    boxes = yolo_boxes_to_corners(box_xy,box_wh)
    scores,boxes,classes = yolo_filter_boxes(box_confidence,boxes,box_class_probs,score_threshold)
    
    boxes = scale_boxes(boxes,image_shape)
    
    scores,boxes,classes = non_max_suppression(scores,boxes,classes, max_boxes,iou_threshold)
    
    return scores, boxes, classes
    
    

In [97]:
with tf.Session() as test_2:
    
    yolo_outputs = (tf.random_normal([19,19,5,1],mean=1,stddev=4,seed=1),tf.random_normal([19,19,5,2],mean=1,stddev=4,seed=1),
                    tf.random_normal([19,19,5,2],mean=1,stddev=4,seed=1),tf.random_normal([19,19,5,80],mean=1,stddev=4,seed=1)
                    )
    scores,boxes,classes = yolo_eval(yolo_outputs)
    
    print('scores[4]='+str(scores[4].eval()))
    print('boxes[4]='+str(boxes[4].eval()))
    print('calsses[4]='+str(classes[4].eval()))
    print('scores.shape' + str(scores.eval().shape))
    print('boxes.shape' + str(boxes.eval().shape))
    print('classes.shape' + str(classes.eval().shape))
    

scores[4]=134.5264
boxes[4]=[ 1222.7573 -1457.8224  3668.2722 -4373.4673]
calsses[4]=49
scores.shape(10,)
boxes.shape(10, 4)
classes.shape(10,)


In [99]:
sess = K.get_session()

In [121]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.)

ValueError: could not convert string to float: 'version https://git-lfs.github.com/spec/v1\n'

In [122]:
class_names

['version https://git-lfs.github.com/spec/v1',
 'oid sha256:634a1132eb33f8091d60f2c346ababe8b905ae08387037aed883953b7329af84',
 'size 625']

In [118]:
yolo_model = load_model("model_data/yolo.h5")

OSError: Unable to open file (unable to open file: name = 'model_data/yolo.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)